In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sb
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
product_cols = [
#         'ind_ahor_fin_ult1','ind_aval_fin_ult1',
        'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
       'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1'
        ]

In [ ]:
months = ['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28', '2015-05-28',
         '2015-06-28', '2015-07-28', '2015-08-28', '2015-09-28', '2015-10-28',
         '2015-11-28', '2015-12-28', '2016-01-28', '2016-02-28', '2016-03-28',
         '2016-04-28', '2016-05-28']

In [ ]:
dates = pd.read_csv('/kaggle/input/santander-pr/train.csv', usecols=['fecha_dato'], header=0)

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
data_new = pd.read_csv('/kaggle/input/santander-pr/train.csv', dtype={"ind_ahor_fin_ult1":"int8",'ind_aval_fin_ult1':"int8", 'ind_cco_fin_ult1':"int8", 
                                                                  'ind_cder_fin_ult1':"int8", 'ind_cno_fin_ult1':"int8", 'ind_ctju_fin_ult1':"int8", 'ind_ctma_fin_ult1':"int8", 
                                                                  'ind_ctop_fin_ult1':"int8", 'ind_ctpp_fin_ult1':"int8",
                                                                  'ind_nomina_ult1':"float16", 'ind_nom_pens_ult1':"float16",
                                                                  'ind_actividad_cliente':"float16", 'cod_prov':"float16",
                                                                  'tipodom':"float16", "indrel":"float16", "ind_nuevo":"float16",
                                                                  'ind_deco_fin_ult1':"int8", 'ind_deme_fin_ult1':"int8", 'ind_dela_fin_ult1':"int8",
                                                                  'ind_ecue_fin_ult1':"int8", 'ind_fond_fin_ult1':"int8", 'ind_hip_fin_ult1':"int8",
                                                                  'ind_plan_fin_ult1':"int8", 'ind_pres_fin_ult1':"int8", 'ind_reca_fin_ult1':"int8", 'ind_tjcr_fin_ult1':"int8", 
                                                                  'ind_valo_fin_ult1':"int8", 'ind_viv_fin_ult1':"int8", 'ind_recibo_ult1':"int8"},
                      parse_dates=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'])
data_test = pd.read_csv('/kaggle/input/santander-pr/test.csv', parse_dates=['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'])

In [ ]:
data_new['Year_Month'] = data_new['fecha_dato'].apply(lambda x: (100 * x.year) + x.month)
data_new.set_index('Year_Month', inplace = True)

In [ ]:
my_list = ["201604"]
data_new = data_new[data_new.index.isin(my_list)]

In [ ]:
data_merged = pd.concat([data_new.assign(ind="train"), data_test.assign(ind="test")], axis=0, sort=False, ignore_index=True)

In [ ]:
del data_new
del data_test

In [ ]:
cols_drop = ['conyuemp', 'ult_fec_cli_1t', 'pais_residencia', 'tipodom', 'nomprov', 'indrel', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1']
data_merged.drop(columns = cols_drop, inplace = True)

In [ ]:
dic=data_merged[['cod_prov','renta']].groupby('cod_prov')['renta'].mean().to_dict()
for key in dic:
    data_merged['renta']=np.where((data_merged['cod_prov']==key) & (data_merged['renta'].isna()),dic.get(key),data_merged['renta'])

In [ ]:
data_merged.loc[data_merged.renta.isnull(),"renta"] = data_merged.loc[data_merged.renta.notnull(),"renta"].median()

In [ ]:
map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            "3.0" : "3",
            "P"   : "P",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            "4.0" : "4",
            "4"   : "4",
            "2"   : "2"}
data_merged['indrel_1mes'].fillna("P",inplace=True)
data_merged['indrel_1mes'] = data_merged['indrel_1mes'].apply(lambda x: map_dict.get(x,x))
data_merged.loc[data_merged['tiprel_1mes'].isnull(),"tiprel_1mes"] = "A"

In [ ]:
data_merged["age"] = pd.to_numeric(data_merged["age"], errors="coerce")
data_merged['antiguedad'] = pd.to_numeric(data_merged['antiguedad'], errors='coerce')

In [ ]:
data_merged.loc[data_merged['age'] < 18,"age"]  = data_merged.loc[(data_merged['age'] >= 18) & (data_merged['age'] <= 30),"age"].mean(skipna=True)
data_merged.loc[data_merged['age'] > 100,"age"] = data_merged.loc[(data_merged['age'] >= 30) & (data_merged['age'] <= 90),"age"].mean(skipna=True)
data_merged["age"].fillna(data_merged["age"].mean(),inplace=True)

In [ ]:
data_merged['canal_entrada'].fillna('KHE', inplace = True)
data_merged['sexo'].fillna('V', inplace = True)
data_merged['segmento'].fillna('02 - PARTICULARES', inplace = True)
data_merged['cod_prov'].fillna(28, inplace = True)

In [ ]:
encoder_dict = {
'sexo'          : {'V':0, 'H':1},
'indrel_1mes'   : {'1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'indresi'       : {-99:0, 'S':1, 'N':2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2},
# 'conyuemp'      : {-99:0, 'N' : 11, 'S' : 13},
'indfall'       : {'N' : 6, 'S' : 1},
'ind_empleado'  : {'N' : 3, 'B': 7, 'F': 9, 'A': 20, 'S': 13},
'tiprel_1mes'   : {'A' : 1, 'I' : 9, 'P' : 6, 'R' : 4, 'N' : 3},
'indext'        : {'N' : 4, 'S' : 9}
}
data_merged.replace(encoder_dict, inplace=True)

In [ ]:
ce_random = [241, 212, 213, 243, 228, 109, 111, 387, 290, 28, 255, 446, 374, 105, 466, 438, 103, 403, 88, 492, 140, 254, 211, 427, 216, 29, 412, 114, 33, 302, 100, 481, 272, 91, 451, 163, 335, 193, 471, 99, 160, 425, 218, 19, 169, 326, 148, 314, 341, 445, 237, 373, 183, 16, 153, 239, 154, 271, 414, 21, 199, 324, 41, 296, 459, 76, 497, 104, 87, 424, 417, 279, 369, 423, 129, 487, 280, 179, 408, 443, 450, 263, 77, 276, 121, 110, 420, 460, 156, 437, 190, 281, 496, 340, 317, 184, 83, 389, 32, 200, 166, 171, 375, 258, 18, 448, 146, 209, 467, 72, 409, 396, 231, 44, 390, 273, 323, 444, 364, 39, 63, 268, 43, 353, 278, 485, 47, 318, 305, 270, 362, 196, 38, 46, 257, 244, 349, 6, 123, 35, 315, 311, 245, 236, 145, 490, 180, 208, 261, 192, 407, 301, 411, 312, 269, 413, 57, 300, 422, 80, 3, 700, 701]
unique_canal_values = list(data_merged['canal_entrada'].unique())
ce_dict = {unique_canal_values[i]: ce_random[i] for i in range(len(unique_canal_values))}
data_merged["canal_entrada"] = data_merged["canal_entrada"].map(ce_dict)

In [ ]:
data_test, data_train = data_merged[data_merged["ind"].eq("test")], data_merged[data_merged["ind"].eq("train")]
data_test.drop(columns = ['ind'], inplace = True)
data_train.drop(columns = ['ind'], inplace = True)

In [ ]:
data_test.reset_index(drop = True, inplace = True)

In [ ]:
def getDataByMonth(dates, molist, clientfeat, prodlist):
    ids = dates.index[dates.fecha_dato.isin(molist)] # dates = fetcha_dato #molist =  particular month
#     print('ids = ', len(ids))
    #print(ids[:10])
    #print(molist)
    usecols = clientfeat + prodlist
    modata = pd.read_csv('../input/santander-pr/train.csv', usecols=usecols,
                skiprows= range(1,ids[0]+1), nrows=len(ids), header=0)
    return modata

In [ ]:
def getLaggedFeatures(mgd, molist):
    #print(mdata.shape)
    #mgd = mdata
    
    for mo in molist:
        print(mo)
        lagmodata = getDataByMonth(dates, [mo], ['ncodpers'], product_cols)
        print(lagmodata.shape)

        #intsec = np.intersect1d(mdata.ncodpers, lagmodata.ncodpers)
        #print(intsec.size)
        #print(np.unique(intsec).size)
        
        print('\nMerging lagged month (' +str(mo) + ')...')
        i = molist.index(mo)
        mgd = pd.merge(mgd, lagmodata, how='left', on='ncodpers', 
                                                    suffixes=[i, i+1])
        print(mgd.shape)
#         print(mgd.info())
        #print(mgd.head())
        #print(mgd.tail())
        print('Filling NaN values with 0')
        mgd.fillna(0, inplace=True)
#         print(mgd.info())
              
#     print(mgd.info())
#     print(mgd.head())
        
    return mgd

In [ ]:
thismonth = ['2016-04-28']
df = pd.DataFrame()
for month in thismonth :    
    lag1month = months[months.index(month) - 1]
    lag2month = months[months.index(month) - 2]
    lag3month = months[months.index(month) - 3]
    lag4month = months[months.index(month) - 4]
    lagmonths = [lag1month, lag2month, lag3month, lag4month]
    df = pd.concat([df, getLaggedFeatures(data_train[data_train['fecha_dato'] == month], lagmonths)])

In [ ]:
data_train = df
del df

In [ ]:
thismonth = ['2016-05-28']
df = pd.DataFrame()
for month in thismonth :    
    lag1month = months[months.index(month) - 1]
    lag2month = months[months.index(month) - 2]
    lag3month = months[months.index(month) - 3]
    lag4month = months[months.index(month) - 4]
    lagmonths = [lag1month, lag2month, lag3month, lag4month]
    df = pd.concat([df, getLaggedFeatures(data_test[data_test['fecha_dato'] == month], lagmonths)])

In [ ]:
data_test = df
del df

In [ ]:
data_test.columns

In [ ]:
data_train['fecha_alta'] = pd.to_datetime(data_train['fecha_alta'], errors='coerce')

In [ ]:
import datetime as dt

data_train['fecha_dato']=data_train['fecha_dato'].map(dt.datetime.toordinal)
data_train['fecha_alta']=data_train['fecha_alta'].map(dt.datetime.toordinal)

In [ ]:
data_test['fecha_dato']=data_test['fecha_dato'].map(dt.datetime.toordinal)
data_test['fecha_alta']=data_test['fecha_alta'].map(dt.datetime.toordinal)

In [ ]:
# li = data_train.columns
# for lis in li:
#     print("'" +lis+"',")

In [ ]:
product_cols = [
#        'ind_ahor_fin_ult10', 'ind_aval_fin_ult10', 
       'ind_cco_fin_ult10', 'ind_cder_fin_ult10',
       'ind_cno_fin_ult10', 'ind_ctju_fin_ult10', 'ind_ctma_fin_ult10',
       'ind_ctop_fin_ult10', 'ind_ctpp_fin_ult10', 'ind_deco_fin_ult10',
       'ind_deme_fin_ult10', 'ind_dela_fin_ult10', 'ind_ecue_fin_ult10',
       'ind_fond_fin_ult10', 'ind_hip_fin_ult10', 'ind_plan_fin_ult10',
       'ind_pres_fin_ult10', 'ind_reca_fin_ult10', 'ind_tjcr_fin_ult10',
       'ind_valo_fin_ult10', 'ind_viv_fin_ult10', 'ind_nomina_ult10',
       'ind_nom_pens_ult10', 'ind_recibo_ult10'
        ]
print(len(product_cols))

In [ ]:
feature_cols = ['fecha_dato', 'ncodpers', 'ind_empleado', 'sexo', 'age', 'fecha_alta',
       'ind_nuevo', 'antiguedad', 'indrel_1mes', 'tiprel_1mes', 'indresi',
       'indext', 'canal_entrada', 'indfall', 'cod_prov',
       'ind_actividad_cliente', 'renta', 'segmento',
                
       'ind_cco_fin_ult11', 'ind_cder_fin_ult11', 'ind_cno_fin_ult11', 'ind_ctju_fin_ult11', 'ind_ctma_fin_ult11', 'ind_ctop_fin_ult11', 'ind_ctpp_fin_ult11', 'ind_deco_fin_ult11',
                'ind_deme_fin_ult11', 'ind_dela_fin_ult11', 'ind_ecue_fin_ult11', 'ind_fond_fin_ult11', 'ind_hip_fin_ult11', 'ind_plan_fin_ult11', 'ind_pres_fin_ult11', 
                'ind_reca_fin_ult11', 'ind_tjcr_fin_ult11', 'ind_valo_fin_ult11', 'ind_viv_fin_ult11', 'ind_nomina_ult11', 'ind_nom_pens_ult11', 'ind_recibo_ult11', 
                'ind_cco_fin_ult12', 'ind_cder_fin_ult12', 'ind_cno_fin_ult12', 'ind_ctju_fin_ult12', 'ind_ctma_fin_ult12', 'ind_ctop_fin_ult12', 'ind_ctpp_fin_ult12', 
                'ind_deco_fin_ult12', 'ind_deme_fin_ult12', 'ind_dela_fin_ult12', 'ind_ecue_fin_ult12', 'ind_fond_fin_ult12', 'ind_hip_fin_ult12', 'ind_plan_fin_ult12', 
                'ind_pres_fin_ult12', 'ind_reca_fin_ult12', 'ind_tjcr_fin_ult12', 'ind_valo_fin_ult12', 'ind_viv_fin_ult12', 'ind_nomina_ult12', 'ind_nom_pens_ult12', 
                'ind_recibo_ult12', 'ind_cco_fin_ult13', 'ind_cder_fin_ult13', 'ind_cno_fin_ult13', 'ind_ctju_fin_ult13', 'ind_ctma_fin_ult13', 'ind_ctop_fin_ult13',
                'ind_ctpp_fin_ult13', 'ind_deco_fin_ult13', 'ind_deme_fin_ult13', 'ind_dela_fin_ult13', 'ind_ecue_fin_ult13', 'ind_fond_fin_ult13', 'ind_hip_fin_ult13',
                'ind_plan_fin_ult13', 'ind_pres_fin_ult13', 'ind_reca_fin_ult13', 'ind_tjcr_fin_ult13', 'ind_valo_fin_ult13', 'ind_viv_fin_ult13', 'ind_nomina_ult13',
                'ind_nom_pens_ult13', 'ind_recibo_ult13', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 
                'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 
                'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
                'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'
       
        ]
print(len(feature_cols))

In [ ]:
data_test.drop(columns = product_cols, inplace = True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train[['renta', 'fecha_alta', 'fecha_dato']] = scaler.fit_transform(data_train[['renta', 'fecha_alta', 'fecha_dato']])
data_test[['renta', 'fecha_alta', 'fecha_dato']] = scaler.fit_transform(data_test[['renta', 'fecha_alta', 'fecha_dato']])

In [ ]:
X = data_train.drop(columns = product_cols, inplace = False)
y = data_train.drop(columns = feature_cols, inplace = False)

In [ ]:
print(X.shape)
print(data_test.shape)

In [ ]:
X['tiprel_1mes'].unique()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from collections import defaultdict
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier

models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = data_test['ncodpers'].values
for c in product_cols:
    print(c)        
#     clf = LogisticRegression(random_state = 42, verbose = 1, n_jobs = -1)
    clf = xgb.XGBClassifier(
            objective = 'binary:logistic',
            eta = 0.08 ,
            max_depth  = 7, 
            silent  = 1, 
            eval_metric = "map@5", 
            min_child_weight = 1,
            subsample = 0.9,
            colsample_bytree = 0.9,
            seed = 24,
            gamma = 0.15, 
            reg_alpha=0.075,
            n_jobs = -1,
            n_estimators = 70
            )
    
    clf.fit(X, y[c])
    p_train1 = clf.predict_proba(data_test)[:,1]
        
    models[c] = clf
    model_preds[c] = p_train1
        
    for id, p in zip(ids, p_train1):
        id_preds[id].append(p)

In [ ]:
data_train.columns

In [ ]:
feature_cols = [
       'fecha_dato', 'ind_empleado', 'sexo', 'age', 'fecha_alta',
       'ind_nuevo', 'antiguedad', 'indrel_1mes', 'tiprel_1mes', 'indresi',
       'indext', 'canal_entrada', 'indfall', 'cod_prov',
       'ind_actividad_cliente', 'renta', 'segmento', 
#         'ind_ahor_fin_ult11', 'ind_aval_fin_ult11',
       'ind_cco_fin_ult11', 'ind_cder_fin_ult11', 'ind_cno_fin_ult11', 'ind_ctju_fin_ult11', 'ind_ctma_fin_ult11', 'ind_ctop_fin_ult11', 'ind_ctpp_fin_ult11', 'ind_deco_fin_ult11',
                'ind_deme_fin_ult11', 'ind_dela_fin_ult11', 'ind_ecue_fin_ult11', 'ind_fond_fin_ult11', 'ind_hip_fin_ult11', 'ind_plan_fin_ult11', 'ind_pres_fin_ult11', 
                'ind_reca_fin_ult11', 'ind_tjcr_fin_ult11', 'ind_valo_fin_ult11', 'ind_viv_fin_ult11', 'ind_nomina_ult11', 'ind_nom_pens_ult11', 'ind_recibo_ult11', 
                'ind_cco_fin_ult12', 'ind_cder_fin_ult12', 'ind_cno_fin_ult12', 'ind_ctju_fin_ult12', 'ind_ctma_fin_ult12', 'ind_ctop_fin_ult12', 'ind_ctpp_fin_ult12', 
                'ind_deco_fin_ult12', 'ind_deme_fin_ult12', 'ind_dela_fin_ult12', 'ind_ecue_fin_ult12', 'ind_fond_fin_ult12', 'ind_hip_fin_ult12', 'ind_plan_fin_ult12', 
                'ind_pres_fin_ult12', 'ind_reca_fin_ult12', 'ind_tjcr_fin_ult12', 'ind_valo_fin_ult12', 'ind_viv_fin_ult12', 'ind_nomina_ult12', 'ind_nom_pens_ult12', 
                'ind_recibo_ult12', 'ind_cco_fin_ult13', 'ind_cder_fin_ult13', 'ind_cno_fin_ult13', 'ind_ctju_fin_ult13', 'ind_ctma_fin_ult13', 'ind_ctop_fin_ult13',
                'ind_ctpp_fin_ult13', 'ind_deco_fin_ult13', 'ind_deme_fin_ult13', 'ind_dela_fin_ult13', 'ind_ecue_fin_ult13', 'ind_fond_fin_ult13', 'ind_hip_fin_ult13',
                'ind_plan_fin_ult13', 'ind_pres_fin_ult13', 'ind_reca_fin_ult13', 'ind_tjcr_fin_ult13', 'ind_valo_fin_ult13', 'ind_viv_fin_ult13', 'ind_nomina_ult13',
                'ind_nom_pens_ult13', 'ind_recibo_ult13', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 
                'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 
                'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
                'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'
        ]
data_train.drop(columns = feature_cols, inplace = True)

In [ ]:
already_active = {}
already_inactive = {}
for row in data_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(tuple(product_cols), row) if c[1] > 0]
    inactive = [c[0] for c in zip(tuple(product_cols), row) if c[1] == 0]
    already_active[id] = active
    already_inactive[id] = inactive

In [ ]:
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    added_preds = [i for i in sorted([i for i in zip(tuple(product_cols), p) if i[0] not in already_active.get(id, [])],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]]
    discontinued_preds = [i for i in sorted([i for i in zip(tuple(product_cols), [1 - x for x in p]) if i[0] not in already_inactive.get(id, [])],
                                  key=lambda i:i [1], 
                                  reverse=True)[:5]]
    total_preds = added_preds + discontinued_preds
    total_preds.sort(key=lambda x: x[1], reverse = True)
    total_preds = [x[0].strip('0') for x in total_preds][:5]
    train_preds[id] = total_preds

In [ ]:
test_preds = []
for row in data_test.values:
    id = row[1] #check the column number before executing for getting ncodpers
    p = train_preds[id]
    test_preds.append(' '.join(p))

In [ ]:
submission = pd.DataFrame({'ncodpers':data_test['ncodpers'],'changed':test_preds})
submission.to_csv('AbbToScoreBadhJaBhai_JULY_ONE.csv', index=False)

In [ ]:
submission

In [ ]:
from IPython.display import FileLink
FileLink(r'AbbToScoreBadhJaBhai.csv')